In [1]:
import altair as alt
import pandas as pd
from vega_datasets import data

pd.set_option("display.max_columns", None)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Create Helper for Saving Outputs

In [2]:
def save_chart(chart, filename, fmt="svg", scale=2):
    filepath = f"charts/{filename}.{fmt}"
    chart.save(filepath, format=fmt, scale=scale)

## Load & Prep Data

In [3]:
arrests_df = pd.read_excel("data/arrests-latest.xlsx")

In [4]:
# roll up duplicates
# Asked AI: how to keep the first value of duplicate rows in pandas and it gave this: df.drop_duplicates(subset=["A"], keep="first")
arrests_dedupe = arrests_df.drop_duplicates(subset=["unique_identifier"], keep="first")
# print(len(arrests_df)) # 291722
# print(len(arrests_dedupe)) # 274017

In [28]:
arrests_april25 = arrests_dedupe[
    (
        (arrests_dedupe["apprehension_date"].dt.month == 4)
        & (arrests_dedupe["apprehension_date"].dt.year == 2025)
    )
].copy()
arrests_april25["apprehension_state"].value_counts()

apprehension_state
TEXAS                          2911
FLORIDA                        1722
CALIFORNIA                      692
GEORGIA                         622
TENNESSEE                       558
VIRGINIA                        541
NEW YORK                        479
ARIZONA                         415
LOUISIANA                       397
ALABAMA                         335
COLORADO                        329
SOUTH CAROLINA                  322
PENNSYLVANIA                    304
NEW JERSEY                      279
NORTH CAROLINA                  277
OKLAHOMA                        263
ARKANSAS                        234
NEVADA                          231
MICHIGAN                        228
INDIANA                         219
MARYLAND                        216
UTAH                            208
ILLINOIS                        205
OHIO                            197
MISSISSIPPI                     180
KENTUCKY                        160
MASSACHUSETTS                   158
KANSAS   

## Create Visualizations

In [5]:
# Disable "..." button on top right of charts
alt.renderers.set_embed_options(actions=False)


# Configure global look (set palette)
# Asked AI: what decorator can i use to set global theme with altair:
# @alt.theme.register("my_global_theme") and gave a sample theme function that I adjusted for my teal theme
@alt.theme.register("teal_theme", enable=True)
def teal_theme():
    return {
        "config": {
            "range": {
                "category": {"scheme": "teals"},
                "heatmap": {"scheme": "teals"},
            },
            "mark": {"color": "#008080"},
        },
    }


In [38]:
# limit df to only needed cols to speed up run time & exclude July 2025 because it's incomplete month
# arrests_date = arrests_dedupe[["apprehension_date"]].copy()
arrests_date = arrests_dedupe[
    ~(
        (arrests_dedupe["apprehension_date"].dt.month == 7)
        & (arrests_dedupe["apprehension_date"].dt.year == 2025)
    )
][["apprehension_date"]].copy()


base = (
    alt.Chart(arrests_date)
    .mark_line()
    .encode(
        alt.X(
            "yearmonth(apprehension_date):T",
            axis=alt.Axis(
                title="Date (Month & Year)",
                tickCount="month",  # ensures monthly tick marks
                format="%b %Y",  # e.g. Jan 2024, Feb 2024
                labelAngle=-45,  # slanted labels for readability
            ),
        ).title("Date"),
        alt.Y("count():Q").title("Total Monthly Arrests"),
    )
    .properties(width=400)
)


rule = (
    alt.Chart(
        pd.DataFrame(
            {
                "apprehension_date": [pd.Timestamp("2025-01-20")],
                "label": ["Trump Inaugurated"],
            }
        )
    )
    .mark_rule(color="red", strokeWidth=2)
    .encode(x="yearmonth(apprehension_date):T", size=alt.value(2))
)

text = (
    alt.Chart(
        pd.DataFrame(
            {
                "apprehension_date": [pd.Timestamp("2025-01-20")],
                "y": [arrests_date["apprehension_date"].value_counts().max()],
                "label": ["Trump Inaugurated"],
            }
        )
    )
    .mark_text(align="left", dx=5, dy=-5, color="red")
    .encode(x="yearmonth(apprehension_date):T", y=alt.value(0), text="label")
)

chart = (base + rule + text).properties(title="ICE Arrests (Sept 2023 - June 2025)")
save_chart(chart, "arrests_over_time")

In [8]:
# edit using this: https://altair-viz.github.io/gallery/groupby-map.htmls
## look into better source for state centroids than using capitals
# see if you can do a version of this that normalizes for population (need secondary dataset)

arrest_states = arrests_dedupe.copy()

# Clean state names and aggregate to counts
# Drop missing / unknowns
arrest_states = arrest_states.dropna(subset=["apprehension_state"])

# Normalize to Title Case to match vega_datasets' "state" names
# (eg 'SOUTH CAROLINA' -> 'South Carolina')
arrest_states["state"] = (
    arrest_states["apprehension_state"].astype(str).str.strip().str.title()
)

# Fix small title-case quirks (e.g. 'Of' in DC)
arrest_states["state"] = arrest_states["state"].replace(
    {
        "District Of Columbia": "District of Columbia",
    }
)

# drop missing values
df = arrest_states.dropna(subset=["state"])

# Count arrests by state
by_state = (
    df.groupby("state", as_index=False).size().rename(columns={"size": "arrests"})
)

# Get map & centroids from vega_datasets
# Base US states topology
states = alt.topo_feature(data.us_10m.url, "states")

# State "centroids": use the state capital coordinates (proxy)
capitals = pd.DataFrame(data.us_state_capitals())  # columns: state, city, lat, lon

centroids = capitals[["state", "lat", "lon"]].copy()

# Join your counts to centroids
plot_df = pd.merge(centroids, by_state, on="state", how="inner")

# Background map
background = (
    alt.Chart(states)
    .mark_geoshape(fill="#eeeeee", stroke="#ffffffb0")
    .project("albersUsa")
)

# Proportional symbols
points = (
    alt.Chart(plot_df)
    .mark_circle(opacity=0.7)
    .encode(
        longitude="lon:Q",
        latitude="lat:Q",
        size=alt.Size("arrests:Q", title="Arrests", scale=alt.Scale(range=[0, 2500])),
    )
    .project("albersUsa")
)


chart = (background + points).properties(
    title="ICE Arrests by State (Sept 2023 - July 2025)", width=500, height=400
)

save_chart(chart, "arrests_by_state")


### Demographics

In [9]:
# departure country: Might scrap -- not very interesting
arrests_departure = arrests_dedupe[["departure_country"]].copy()
top5 = (
    arrests_departure["departure_country"]
    .value_counts()
    .nlargest(5)
    .rename_axis("departure_country")
    .reset_index(name="arrests")
)

# create chart
chart = (
    alt.Chart(top5)
    .mark_bar()
    .encode(
        alt.X("departure_country:N", sort="-y").title("Departure Country"),
        alt.Y("arrests:Q").title("Arrests"),
    )
)

save_chart(chart, "departure_country")


In [40]:
arrests_dedupe.head(3)

,apprehension_date,apprehension_state,apprehension_aor,final_program,final_program_group,apprehension_method,apprehension_criminality,case_status,case_category,departed_date,departure_country,final_order_yes_no,final_order_date,birth_year,citizenship_country,gender,apprehension_site_landmark,unique_identifier,apprehension_date_time,duplicate_likely,file_original,sheet_original,row_original
0,2023-09-01,CALIFORNIA,San Francisco Area of Responsibility,ERO Criminal Alien Program,ICE,CAP Federal Incarceration,1 Convicted Criminal,6-Deported/Removed - Deportability,[16] Reinstated Final Order,2023-09-02,MEXICO,YES,2001-04-09,1972,MEXICO,Male,"FRE GENERAL AREA, NON-SPECIFIC",3ddc9dfa23c14851a7cd709ad2e6c52a4e36a08b,2023-09-01 00:00:00,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,69540
1,2023-09-01,SOUTH CAROLINA,Atlanta Area of Responsibility,ERO Criminal Alien Program,ICE,CAP Local Incarceration,2 Pending Criminal Charges,8-Excluded/Removed - Inadmissibility,[8C] Excludable / Inadmissible - Administrativ...,2024-01-17,HONDURAS,YES,2023-11-21,1994,HONDURAS,Male,"RICHLAND COUNTY, SC",8b3087d9852e9db2203541ebb2fc90826c74a647,2023-09-01 00:00:00,0.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,156756
2,2023-09-01,NaN,NaN,Alternatives to Detention,ICE,ERO Reprocessed Arrest,3 Other Immigration Violator,E-Charging Document Canceled by ICE,[8A] Excludable / Inadmissible - Hearing Not C...,NaT,NaN,NO,NaT,1966,ECUADOR,Male,NaN,db5178743c5753e8acf6f2f2eca88af68395073f,2023-09-01 00:19:00,1.0,2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arre...,Admin Arrests,247560


In [43]:
# limit to values where origin and departure country differ

# drop missing values
diff_departures = (arrests_dedupe.copy()).dropna(
    subset=["citizenship_country", "departure_country"]
)

diff_departures.nunique()


apprehension_date                697
apprehension_state                60
apprehension_aor                  26
final_program                     14
final_program_group                1
apprehension_method               26
apprehension_criminality           3
case_status                        6
case_category                     25
departed_date                    714
departure_country                191
final_order_yes_no                 2
final_order_date                9525
birth_year                        85
citizenship_country              177
gender                             3
apprehension_site_landmark      4491
unique_identifier             156178
apprehension_date_time        133522
duplicate_likely                   2
file_original                      1
sheet_original                     1
row_original                  156179
dtype: int64

In [39]:
# prep df
arrests_citizen = arrests_dedupe[["citizenship_country"]].copy()
top_n = (
    arrests_citizen["citizenship_country"]
    .value_counts()
    .nlargest(10)
    .rename_axis("citizenship_country")
    .reset_index(name="arrests")
)

# create chart
chart = (
    alt.Chart(top_n)
    .mark_bar()
    .encode(
        alt.X("citizenship_country:N", sort="-y", axis=alt.Axis(labelAngle=-45)).title(
            "Citizenship Country"
        ),
        alt.Y("arrests:Q").title("Arrests"),
    )
).properties(title="Top 10 Countries of Origin")

save_chart(chart, "citizenship_country")

In [35]:
# top5 citizenship country arrests make up X% of total arrests

# prep data
total = arrests_dedupe.shape[0]
top_n_sum = top_n["arrests"].sum()
top_n_perc = (top_n_sum / total) * 100
remaining_perc = ((total - top_n_sum) / total) * 100

# Asked AI: whats the syntax for creating pandas df and it gave the following:
# data = {
#     "name": ["Alice", "Bob", "Charlie"],
#     "age": [25, 30, 35],
#     "city": ["NYC", "LA", "Chicago"]
# }

# df = pd.DataFrame(data)

pie_data = pd.DataFrame(
    {
        "category": ["Top 5 Countries", "All Other Countries"],
        "arrest_perc": [top_n_perc, remaining_perc],
    }
)

# create chart
chart = (
    alt.Chart(pie_data)
    .mark_arc()
    .encode(
        alt.Theta("arrest_perc:Q", title="Percent of Total Arrests"),
        alt.Color("category:N", title="Citizenship Country"),
    )
    .properties(
        title="Share of Total Arrests by Citizenship Country: Top 5 vs. All Other"
    )
)

save_chart(chart, "pie_top5_countries")

In [12]:
methods = arrests_dedupe[["apprehension_method", "apprehension_date"]].copy()


# bucket apprehension methods
bucket_map = {
    "CAP Local Incarceration": "CAP",
    "CAP Federal Incarceration": "CAP",
    "CAP State Incarceration": "CAP",
    "Non-Custodial Arrest": "ERO",
    "Custodial Arrest": "ERO",
    "ERO Reprocessed Arrest": "ERO",
    "Other efforts": "Task Force / Other Agency",
    "Other Task Force": "Task Force / Other Agency",
    "Other Agency (turned over to INS)": "Task Force / Other Agency",
    "Law Enforcement Agency Response Unit": "Task Force / Other Agency",
    "Organized Crime Drug Enforcement Task Force": "Task Force / Other Agency",
    "287(g) Program": "287g / Probation",
    "Probation and Parole": "287g / Probation",
    "Patrol Border": "Border / Patrol",
    "Patrol Interior": "Border / Patrol",
    "Located": "Border / Patrol",
    "Worksite Enforcement": "Worksite / Inspections",
    "Inspections": "Worksite / Inspections",
    "Anti-Smuggling": "Worksite / Inspections",
    "Traffic Check": "Worksite / Inspections",
    "Transportation Check Aircraft": "Transportation",
    "Transportation Check Bus": "Transportation",
    "Transportation Check Passenger Train": "Transportation",
    "Presented During Inspection": "Transportation",
    "Boat Patrol": "Transportation",
    "Crewman/Stowaway": "Transportation",
    "Criminal Alien Program": "Miscellaneous",
}

# asked AI: how to extract year from date column in pandas: df["year"] = df["date"].dt.year
methods["year"] = methods["apprehension_date"].dt.year
methods["method_bucket"] = (
    methods["apprehension_method"].map(bucket_map).fillna("Miscellaneous")
)

# group by method and arrest year
rolled = methods.groupby(["method_bucket", "year"]).size().reset_index(name="count")

chart = (
    alt.Chart(rolled)
    .mark_rect()
    .encode(
        alt.X("year:N", title="Arrest Year"),
        alt.Y("method_bucket:N", title="Apprehension Method"),
        alt.Color("count:Q", title="Arrest Count"),
    )
    .properties(width=200)
)

save_chart(chart, "heatmap_method")

In [13]:
# Age Distribution (histogram)
age_df = arrests_dedupe.copy()
# create age col
age_df["age"] = 2025 - age_df["birth_year"]
age_df = age_df[["age"]]

base = alt.Chart(age_df)

bar = base.mark_bar().encode(
    alt.X("age:Q", bin=True, axis=alt.Axis(title="Age (years)")),
    alt.Y("count()", axis=alt.Axis(title="Count of Arrests")),
)

# Mean line (red)
rule = base.mark_rule(color="red", strokeWidth=5).encode(x="mean(age):Q")

# Label for mean line
label = base.mark_text(
    align="left",  # position to the right of the line
    dx=5,  # horizontal offset
    dy=-5,  # vertical offset (move up)
    color="red",
).encode(
    x="mean(age):Q",
    y=alt.value(0),  # y=0 or small positive value keeps it near x-axis
    text=alt.value("Mean Age"),
)

chart = (bar + rule + label).properties(title="Distribution of Arrests by Age")

save_chart(chart, "age_histogram")


In [14]:
# time from arrest to deportation (subset of data: only deported persons)
# print(arrests_dedupe["final_order_date"].isnull().sum()) # 107250
# print(len(arrests_dedupe)) # 274017
# ~1/3 final_order_date values are missing, exclude from analysis and make a note on chart

final_date_df = (arrests_dedupe.copy()).dropna(
    subset=["departed_date", "apprehension_date"]
)

final_date_df["days_to_departure"] = (
    final_date_df["departed_date"] - final_date_df["apprehension_date"]
).dt.days.astype("Int64")
len(final_date_df[final_date_df["days_to_departure"] > 0])
# final_date_df[final_date_df["days_to_departure"] < 0]

# final_date_df["decision_days"].mean()


147268

In [15]:
yes_count = (final_date_df["final_order_yes_no"] == "YES").sum()
no_count = (final_date_df["final_order_yes_no"] == "NO").sum()
total = (final_date_df["final_order_yes_no"]).count()

pie_deportations = pd.DataFrame(
    {
        "deportation_status": ["Yes", "No"],
        "perc": [(yes_count / total), (no_count / total)],
    }
)

chart = (
    alt.Chart(pie_deportations)
    .mark_arc()
    .encode(
        alt.Theta("perc:Q", title="Percent of Total Arrests"),
        alt.Color("deportation_status:N", title="Deportation Status"),
    )
    .properties(title="Share of Deportations")
)  # Or "Share of Arrests That Led To Deportation"

save_chart(chart, "pie_deportations")

In [17]:
# Density: time taken between arrest and deportation
# limit to only deported arrests & "days_to_departure" > 0
deported_df = final_date_df[
    (final_date_df["final_order_yes_no"] == "YES")
    & (final_date_df["days_to_departure"] > 0)
]

mean_days = deported_df["days_to_departure"].mean()

# create chart
base = (
    alt.Chart(deported_df)
    .transform_density(
        "days_to_departure",
        as_=["days_to_departure", "density"],
    )
    .mark_area()
    .encode(
        alt.X("days_to_departure:Q", title="Number of Days"),
        alt.Y("density:Q", title="Density"),
    )
)

mean_line = (
    alt.Chart(pd.DataFrame({"days_to_departure": [mean_days]}))
    .mark_rule(color="red", strokeWidth=2)
    .encode(x="days_to_departure:Q")
)

mean_label = (
    alt.Chart(
        pd.DataFrame(
            {
                "days_to_departure": [mean_days],
                "label": [f"Mean = {mean_days:.0f} days"],
            }
        )
    )
    .mark_text(
        align="left",
        angle=0,
        dx=5,  # move right of the line
        dy=5,  # move slightly up
        color="red",
    )
    .encode(
        x="days_to_departure:Q",
        y=alt.value(0.018),  # roughly top of the chart
        text="label",
    )
)

chart = (base + mean_line + mean_label).properties(
    title="Density of Days Between Arrest and Departure", width=400
)

save_chart(chart, "density_days")